# Forecasting #

The longer you look back, the farther you can look forward. (Winston Churchill) 

Forecastig of the development of stocks is quite a challagning task with uncertain outcome. In these kind of situations 
it is of advantage to use scenarios to identify possible outcomes.

- Forecasting applying existing history data
- Forecasting of Global Trend (Bull or Bear)
- Forecasting of Crash
- Forecating of Individual Stocks Short Term

### Setup ###

In [1]:
%classpath config resolver maven-public http://192.168.1.10:8081/repository/maven-public/
%classpath add mvn ch.pschatzmann:investor:0.9-SNAPSHOT
%classpath add mvn ch.pschatzmann:jupyter-jdk-extensions:0.0.1-SNAPSHOT


Added new repo: maven-public


In [16]:
// our stock evaluation framwork
import ch.pschatzmann.dates._;
import ch.pschatzmann.stocks._;
import ch.pschatzmann.stocks.data.universe._;
import ch.pschatzmann.stocks.input._;
import ch.pschatzmann.stocks.accounting._;
import ch.pschatzmann.stocks.accounting.kpi._;
import ch.pschatzmann.stocks.execution._;
import ch.pschatzmann.stocks.execution.fees._;
import ch.pschatzmann.stocks.execution.price._;
import ch.pschatzmann.stocks.parameters._;
import ch.pschatzmann.stocks.strategy._;
import ch.pschatzmann.stocks.strategy.optimization._;
import ch.pschatzmann.stocks.strategy.allocation._;
import ch.pschatzmann.stocks.strategy.selection._;
import ch.pschatzmann.stocks.integration._;
import ch.pschatzmann.stocks.forecasting._;
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._;

// java
import java.util.stream.Collectors;
import java.util._;
import java.lang._;
import java.util.function.Consumer;

// jupyter custom displayer
import ch.pschatzmann.display.Displayers
import ch.pschatzmann.display.Table


import ch.pschatzmann.dates._
import ch.pschatzmann.stocks._
import ch.pschatzmann.stocks.data.universe._
import ch.pschatzmann.stocks.input._
import ch.pschatzmann.stocks.accounting._
import ch.pschatzmann.stocks.accounting.kpi._
import ch.pschatzmann.stocks.execution._
import ch.pschatzmann.stocks.execution.fees._
import ch.pschatzmann.stocks.execution.price._
import ch.pschatzmann.stocks.parameters._
import ch.pschatzmann.stocks.strategy._
import ch.pschatzmann.stocks.strategy.optimization._
import ch.pschatzmann.stocks.strategy.allocation._
import ch.pschatzmann.stocks.strategy.selection._
import ch.pschatzmann.stocks.integration._
import ch.pschatzmann.stocks.forecasting._
import ch.pschatzmann.stocks.strategy.OptimizedStrategy.Schedule._
import java.util.stream.Collectors
import j...

In [30]:
Displayers.setup("WARN")

Context.setCacheActive(false);
Context.isCacheActive();


false

## Forecasting for AAPL ##

In [45]:
var aapl = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());

Table.create(aapl)

In [38]:
new SimpleTimePlot {
    data = Table.create(aapl).seq()
    columns = Seq("AAPL.Open","AAPL.Closing")
    showLegend = false
}

## Simulation of Future using the Past ##


One possiblity is to simulate the future based on the development of the past: Looking at the chart we can e.g.
investigate how our strategy would behave if we would be in a simialr position like on March 2015.

We simulate the future develpment based on the development starting from
- 2012-09-01 of the AAPL stock
- 2015-03-01 of the AAPL stock
- 2017-03-01 of the NYSE Composite Index 


In [46]:
var nya = new StockData(new StockID("^NYA", "INDEX"), new MarketArchiveHttpReader());

var aapl = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var aaplWithFuture1 = new ForecasterFromSimulationOnHistory(Context.date("2012-09-01")).forecast(aapl, 200)
var aaplWithFuture2 = new ForecasterFromSimulationOnHistory(Context.date("2015-03-01")).forecast(aapl, 200)
var aaplWithFuture3 = new ForecasterFromSimulationOnHistory(Context.date("2017-03-01"),nya).forecast(aapl, 200)

// display chart
var chart = new TimeSeriesChart()
var startDisplayDate = Context.date("2012-01-01")
chart.add(aaplWithFuture1.getHistory(startDisplayDate),"1)2012-09-01")
chart.add(aaplWithFuture2.getHistory(startDisplayDate),"2)2015-03-01")
chart.add(aaplWithFuture3.getHistory(startDisplayDate),"3)2017-03-01 nya")

new SimpleTimePlot {
    data = Table.create(aaplWithFuture1).seq()
    columns = Seq("AAPL.Open","AAPL.Closing")
    showLegend = false
}

<console>: 164

### Result for Worst Case Option 1 ###

In [21]:
var dates = Context.getDateRanges("2018-01-01","2019-01-01");

var account1 = new Account("Simulation","USD", 100000.00, dates.get(0).getStart(), new PerTradeFees(10.0));
var aapl = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var aaplWithFuture1 = new ForecasterFromSimulationOnHistory(Context.date("2012-09-01")).forecast(aapl, 200)
var strategy1 = new RSI2Strategy(aaplWithFuture1);
var trader1 = new PaperTrader(account1);
var state = new Fitness(trader1).getFitness(strategy1, dates.get(0));

state.result()

AbsoluteReturnAvaragePerDay:-64; NumberOfSells:1; AbsoluteReturn:-12080; ReturnPercentStdDev:0; MaxDrawDownPercent:22736; AbsoluteReturnStdDev:900; ReturnPercentAnualized:-16; NumberOfTradedStocks:1; NumberOfCashTransfers:1; ActualValue:87920; NumberOfTrades:2; MaxDrawDownHighValue:101144; SharpeRatio:-1; MaxDrawDownNumberOfDays:158; UnrealizedGains:0; MaxDrawDownLowValue:78408; RealizedGains:-12060; TotalFees:20; PurchasedValue:87920; ReturnPercent:-12; Cash:87920; NumberOfBuys:1; 

In [40]:
Table.create(account1.getTransactions())


###  Result for Option 2 ###

In [23]:
var dates = Context.getDateRanges("2018-01-01","2019-01-01");

var account2 = new Account("Simulation","USD", 100000.00, dates.get(0).getStart(), new PerTradeFees(10.0));
var aapl = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var aaplWithFuture2 = new ForecasterFromSimulationOnHistory(Context.date("2015-03-01")).forecast(aapl, 200)
var strategy2 = new RSI2Strategy(aaplWithFuture2);
var trader2 = new PaperTrader(account2);
var state2 = new Fitness(trader2).getFitness(strategy2, dates.get(0));

state2.result()

AbsoluteReturnAvaragePerDay:-25; NumberOfSells:1; AbsoluteReturn:-4701; ReturnPercentStdDev:0; MaxDrawDownPercent:23743; AbsoluteReturnStdDev:1447; ReturnPercentAnualized:-6; NumberOfTradedStocks:1; NumberOfCashTransfers:1; ActualValue:95299; NumberOfTrades:2; MaxDrawDownHighValue:108853; SharpeRatio:0; MaxDrawDownNumberOfDays:117; UnrealizedGains:0; MaxDrawDownLowValue:85110; RealizedGains:-4681; TotalFees:20; PurchasedValue:95299; ReturnPercent:-5; Cash:95299; NumberOfBuys:1; 

In [41]:
Table.create(account2.getTransactions())


### Result for Best Case - Option 3 ###

In [42]:
var dates = Context.getDateRanges("2018-01-01","2019-01-01");

var account3 = new Account("Simulation","USD", 100000.00, dates.get(0).getStart(), new PerTradeFees(10.0));
var aapl = new StockData(new StockID("AAPL", "NASDAQ"), new MarketArchiveHttpReader());
var aaplWithFuture3 = new ForecasterFromSimulationOnHistory(Context.date("2017-03-01"),nya).forecast(aapl, 200)
var strategy3 = new RSI2Strategy(aaplWithFuture3);
var trader3 = new PaperTrader(account3);
var state3 = new Fitness(trader3).getFitness(strategy3, dates.get(0));

state3.result()

<console>: 158

In [26]:
Displayers.display(account3.getTransactions().collect(Collectors.toList()));


<console>: 151

## Arima ##

In [43]:
import ch.pschatzmann.stocks.forecasting._
import com.workday.insights.timeseries.arima.struct.ArimaParams

var aapl = new StockData(new StockID("^NYA", "INDEX"), new MarketArchiveHttpReader());
var params = new ArimaParams(1, 0, 0, 0, 0, 0, 0);
var arima = new ForecasterARIMA(params)
var aaplArima = arima.forecast(aapl, 800)

// display chart
var chart = new TimeSeriesChart()
chart.add(aaplArima.getHistory(Context.date("2016-01-01")),"AAPL")

chart

<console>: 161

In [28]:
var result = arima.getClose()
println(result.getRMSE())
println(result.getMaxNormalizedVariance)
result.getLog()

<console>: 150

In [29]:
var result = arima.getOpen()
println(result.getRMSE())
println(result.getMaxNormalizedVariance)
result.getLog()

<console>: 150